In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
data = pd.read_csv("/Users/dhritiman/Downloads/2gaussian.txt", sep = ' ', header = None)

In [3]:
data.head()

,0,1
0,7.571044,3.530274
1,7.337218,4.262713
2,3.071828,1.118019
3,6.226851,3.667489
4,3.513142,1.603125


In [4]:
n = len(data)
np.random.seed(3)
random_probabilities = np.random.rand(n,2)

In [5]:
random_probabilities

array([[ 0.5507979 ,  0.70814782],
       [ 0.29090474,  0.51082761],
       [ 0.89294695,  0.89629309],
       ..., 
       [ 0.35917279,  0.83981295],
       [ 0.682137  ,  0.18772618],
       [ 0.36702737,  0.40534278]])

In [6]:
sum_random_probabilities = np.sum(random_probabilities, axis = 1, keepdims = True)

In [7]:
prob_split = random_probabilities/sum_random_probabilities

In [8]:
prob_split

array([[ 0.43750727,  0.56249273],
       [ 0.36284521,  0.63715479],
       [ 0.49906493,  0.50093507],
       ..., 
       [ 0.29956385,  0.70043615],
       [ 0.78418884,  0.21581116],
       [ 0.47519621,  0.52480379]])

In [9]:
# Initializing probability split

cluster1 = prob_split[:,0]
cluster2 = prob_split[:,1]

In [10]:
# Initializing M-Step values

not_converged = True

mean_1 = np.zeros(2)
mean_2 = np.zeros(2)
cov_1 = np.zeros(shape = [2, 2], dtype = float)
cov_2 = np.zeros(shape = [2, 2], dtype = float)

In [11]:
iterations = 0
while not_converged:
    mean_old_1 = mean_1
    mean_old_2 = mean_2
    cov_old_1 = cov_1
    cov_old_2 = cov_2    
    
    ##################################
    # M - step
    sum_cluster1 = np.sum(cluster1)
    sum_cluster2 = np.sum(cluster2)
    
    mean_1 = np.array([np.sum(data[0] * cluster1), np.sum(data[1] * cluster1)])/sum_cluster1
    
    mean_2 = np.array([np.sum(data[0] * cluster2), np.sum(data[1] * cluster2)]/sum_cluster2)
    
    w1 = sum_cluster1/n
    w2 = sum_cluster2/n
    
    cov_1 = np.zeros(shape = [2, 2], dtype = float)
    cov_2 = np.zeros(shape = [2, 2], dtype = float)
    for i in range(n):        
        tmp_1 = np.matrix(data.iloc[[i]] - mean_1)
        cov_1 += cluster1[i] * (tmp_1.T * tmp_1)
        
        tmp_2 = np.matrix(data.iloc[[i]] - mean_2)
        cov_2 += cluster2[i] * (tmp_2.T * tmp_2)
        
    
    cov_1 /= sum_cluster1
    cov_2 /= sum_cluster2
    
    ##################################
    # E - step
    inv_1 = np.linalg.pinv(cov_1)
    inv_2 = np.linalg.pinv(cov_2)
    det_1 = np.linalg.det(cov_1)
    det_2 = np.linalg.det(cov_2)
    
    print("MEAN:", mean_1, mean_2)
    print("COV:", cov_1, cov_2)
    
    prob_1 = []
    prob_2 = []
    for i in range(n):
        tmp = np.matrix(data.iloc[[i]])
        denominator_1 = (2 * np.pi * np.sqrt(det_1))
        prob_1.append(np.exp(-np.sum((tmp - mean_1) * inv_1 * (tmp - mean_1).T)/2)/denominator_1)
        
        denominator_2 = (2 * np.pi * np.sqrt(det_2))
        prob_2.append(np.exp(-np.sum((tmp - mean_2) * inv_2 * (tmp - mean_2).T)/2)/denominator_2)
        
    
    prob_1 = np.array(prob_1)
    prob_2 = np.array(prob_2)

    
    total = []    
    for i in range(n):
        tmp_total = prob_1[i] * w1 + prob_2[i] * w2
        cluster1[i] = prob_1[i] * w1/tmp_total
        cluster2[i] = prob_2[i] * w2/tmp_total
        total.append(tmp_total)
    
    
    if (np.round(cov_old_1, 5) == np.round(cov_1, 5)).all() and (np.round(cov_old_2, 5) == np.round(cov_2, 5)).all() and (np.round(mean_old_1, 5) == np.round(mean_1, 5)).all() and (np.round(mean_old_2, 5) == np.round(mean_2, 5)).all():
        not_converged = False
        
    iterations += 1


('MEAN:', array([ 5.69232331,  3.67940225]), array([ 5.64283645,  3.66343916]))
('COV:', array([[ 4.5452468 ,  1.18048168],
       [ 1.18048168,  1.82227204]]), array([[ 4.62142311,  1.16583708],
       [ 1.16583708,  1.86280083]]))
('MEAN:', array([ 5.71010782,  3.67728643]), array([ 5.62502407,  3.66555805]))
('COV:', array([[ 4.51851336,  1.18964225],
       [ 1.18964225,  1.79653083]]), array([[ 4.64580147,  1.15655823],
       [ 1.15655823,  1.88864071]]))
('MEAN:', array([ 5.74607641,  3.67392716]), array([ 5.58898229,  3.66892191]))
('COV:', array([[ 4.45730384,  1.21501448],
       [ 1.21501448,  1.74987229]]), array([[ 4.69840618,  1.13124334],
       [ 1.13124334,  1.9354484 ]]))
('MEAN:', array([ 5.81742058,  3.66785238]), array([ 5.51722203,  3.67501433]))
('COV:', array([[ 4.32850266,  1.26904588],
       [ 1.26904588,  1.66882735]]), array([[ 4.79513671,  1.07840173],
       [ 1.07840173,  2.01695976]]))
('MEAN:', array([ 5.94313516,  3.66322928]), array([ 5.38791993,  3.

('MEAN:', array([ 7.01317774,  3.983151  ]), array([ 2.9941885 ,  3.05208986]))
('COV:', array([[ 0.97470764,  0.4974412 ],
       [ 0.4974412 ,  1.001118  ]]), array([[ 1.01032791,  0.02718118],
       [ 0.02718118,  2.93777896]]))
('MEAN:', array([ 7.01316819,  3.98314554]), array([ 2.9941701 ,  3.05209205]))
('COV:', array([[ 0.97472429,  0.49745065],
       [ 0.49745065,  1.00112599]]), array([[ 1.0102975 ,  0.0271845 ],
       [ 0.0271845 ,  2.93779325]]))
('MEAN:', array([ 7.01316173,  3.98314185]), array([ 2.99415767,  3.05209353]))
('COV:', array([[ 0.97473554,  0.49745703],
       [ 0.49745703,  1.00113138]]), array([[ 1.01027696,  0.02718673],
       [ 0.02718673,  2.9378029 ]]))
('MEAN:', array([ 7.01315738,  3.98313936]), array([ 2.99414928,  3.05209452]))
('COV:', array([[ 0.97474313,  0.49746134],
       [ 0.49746134,  1.00113502]]), array([[ 1.01026309,  0.02718825],
       [ 0.02718825,  2.93780942]]))
('MEAN:', array([ 7.01315443,  3.98313768]), array([ 2.99414361,  3.